In [1]:
# INITIALIZATION / IMPORTS
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


from IPython.display import display
import pandas as pd

import msticpy.sectools as sectools
import msticpy.nbtools as mas
from msticpy.sectools import VTLookup, IoCExtract
from msticpy.sectools.geoip import GeoLiteLookup, IPStackLookup
from msticpy.nbtools.entityschema import IpAddress, GeoLocation
from msticpy.nbtools.foliummap import FoliumMap

In [2]:
# OPTION 1 : import the key api named 'vt' from the keyring manager
# below the python cli (to execute once) create the api key : 
# >>> import keyring
# >>> import readline
# >>> keyring.set_password(service_name="vt",username="None",password="xxx")
# >>> readline.clear_history()
import keyring
keyring.get_keyring()
vt_apikey = keyring.get_credential("vt", "None")

In [1]:
# OPTION 2 : import the key api named from the environment variable 
import os
vt_apikey = os.environ['VT_API_KEY']

In [ ]:
# STEP 1 / CREATE A VTLOOKUP INSTANCE
vt_lookup = sectools.VTLookup(vt_apikey, verbosity=2)
# How these map to VT lookup types
# VTLookup._VT_TYPE_MAP

In [ ]:
# STEP 2.1 / LOAD AN IP LIST
# CSV format is N columns with at least one column named 'IP' 
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

ip_list = pd.read_csv(filename)
ip_list.drop_duplicates(subset='IP',inplace=True)
#iplist

ioc_extractor = IoCExtract()
vt_lookup = VTLookup(vt_apikey, verbosity=2)
output_df = ioc_extractor.extract(data=ip_list, 
                                  columns=['IP'], 
                                  ioc_types=vt_lookup.supported_ioc_types)
output_df

In [4]:
# STEP 2.2 / ENTER AN IP ADDRESS
import ipywidgets as widgets
ip_addr = widgets.Text(
    value='',
    placeholder='',
    description='IP Address',
    style={'description_width': '178px'},
    disabled=False
)

widgets.VBox([ip_addr])



In [ ]:
# STEP 2.2 / LOAD THE IP ADDRESS 
ip_list = pd.DataFrame([{'IP':ip_addr.value}])
ip_list
ioc_extractor = IoCExtract()
vt_lookup = VTLookup(vt_apikey, verbosity=2)
output_df = ioc_extractor.extract(data=ip_list, 
                                  columns=['IP'], 
                                  ioc_types=vt_lookup.supported_ioc_types)
output_df

In [ ]:
# STEP 3 / Check the reputation of the iplist in VirusTotal
from past.builtins import xrange
vt_lookup = VTLookup(vt_apikey, verbosity=2)
vt_res = [[]]

# Submit a subset of the found IoCs (ignore dns since a lot of )
vt_results = vt_lookup.lookup_iocs(data=output_df, 
                                   type_col='IoCType', 
                                   src_col='Observable')

#clean the DF, deleting uninteresting cols
vt_res = vt_results.drop(columns=['IoCType', 'Status', 'ResponseCode', 'RawResponse', 'Resource', 'SourceIndex', 'VerboseMsg', 'Resource', 'ScanId', 'Permalink', 'MD5', 'SHA1', 'SHA256', 'ResolvedIPs'])


In [ ]:
# STEP 4 / GEOLOCATE THE IP(S) ADDRESS(ES)
iplocation = GeoLiteLookup()
vt_res = iplocation.df_lookup_ip(vt_res,'Observable')
display(vt_res)

folium_map = FoliumMap(location=[vt_res.Latitude.mean(), vt_res.Longitude.mean()], zoom_start=1)
# Set Icon properties to display
icon_props = {"color": "green"}

def create_ip_entity(row):
    ip_ent = IpAddress(Address=row["Observable"])
    geo_loc = create_geo_entity(row)
    ip_ent.Location = geo_loc
    return ip_ent

def create_geo_entity(row):
    # get subset of fields for GeoLocation
    loc_props = row[["CountryCode_x", "CountryName_x", "City_x", "Longitude_x", "Latitude_x"]]
    geo_loc = GeoLocation(**loc_props.to_dict())
    return geo_loc

# Set additional properties to display in tooltips
# in AdditionalData property of IpAddress entities
geo_locs = list(vt_res.apply(create_geo_entity, axis=1).values)
ip_ents = list(vt_res.apply(create_ip_entity, axis=1).values)
folium_map.add_ip_cluster(ip_entities=ip_ents, **icon_props)

# display the map (or just have folium_map as the last statement
# of the cell - Jupyter will use display() automatically)
display(folium_map)
#folium_map.save('result.html')
#os.system('explorer.exe result.html')

In [9]:
#folium_map.save('result.html')